##### This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install tweepy

In [ ]:
#Importing Libraries
import tweepy
from textblob import TextBlob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud
import json
from collections import Counter

In [ ]:
#Part-1: Authorization and Search tweets
#Getting authorization
consumer_key = 'lE9dP8f9h1rhDDXbdXXHdmhq4'
consumer_key_secret = 'ZYYlV7isekB9hHDxpS5YhMuTLujfk2cGZhaYeujzOcW8kxAyDI'
access_token = '1400384678679306241-dFD0y37BQYw8eZUhdDTpkPg9ywvAoo'
access_token_secret = 'uJ3NGLqqXRQGoMyYbCrYp9vaQs9T73TeE0AauApebZbTY'
auth = tweepy.OAuthHandler(consumer_key, consumer_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
db_tweets = pd.DataFrame(columns = ['username', 'acctdesc', 'location', 'following',
                                            'followers', 'totaltweets', 'usercreatedts', 'tweetcreatedts', 
                                            'retweetcount', 'text', 'hashtags'])

In [ ]:
search_words='covid-19'
date_since = "2020-10-03"
numTweets = 2500
numRuns = 6

In [ ]:
tweets = tweepy.Cursor(api.search, q=search_words, lang="id",since=date_since, tweet_mode='extended').items(numTweets)

In [ ]:
tweet_list=(tweet for tweet in tweets)

In [ ]:
noTweets = 0
for tweet in tweet_list:
    #pull the values
        username = tweet.user.screen_name
        acctdesc = tweet.user.description
        location = tweet.user.location
        following = tweet.user.friends_count
        followers = tweet.user.followers_count
        totaltweets = tweet.user.statuses_count
        usercreatedts = tweet.user.created_at
        tweetcreatedts = tweet.created_at
        retweetcount = tweet.retweet_count
        hashtags = tweet.entities['hashtags']
        try:
            text = tweet.retweeted_status.full_text
        except AttributeError: #not a Retweet
                text = tweet.full_text
        ith_tweet = [username, acctdesc, location, following, followers, totaltweets,
                     usercreatedts, tweetcreatedts, retweetcount, text, hashtags]
        db_tweets.loc[len(db_tweets)] = ith_tweet
        noTweets += 1

In [ ]:
db_tweets.head

In [ ]:
db_tweets["location"]

In [ ]:
counts = db_tweets["username"].value_counts()
plt.bar(range(len(counts)), counts)
plt.show()
print(counts)

In [ ]:
counts = db_tweets["location"].value_counts()
plt.bar(range(len(counts)), counts)
plt.show()
print(counts)

In [ ]:
#Cleaning Tweets
corpusx = []
for i in range(0, 1000):
    tweet = re.sub('[^a-zA-Z0-9]', ' ', db_tweets['text'][i])
    tweet = tweet.lower()
    tweet = re.sub('rt', '', tweet)
    tweet = re.sub('http', '', tweet)
    tweet = re.sub('https', '', tweet)
    tweet = tweet.split()
    ps = PorterStemmer()
    tweet = [ps.stem(word) for word in tweet if not word in set(stopwords.words('english'))]
    tweet = ' '.join(tweet)
    corpusx.append(tweet)

In [ ]:
#Part-5: Visualization

#Word Cloud
all_words = ' '.join([text for text in corpusx])
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(all_words)
plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()